In [2]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math as math
import datetime as dt

# import seaborn as sns
import pandas as pd
import geopandas as gpd
from sodapy import Socrata
import plotly.figure_factory as ff

%load_ext lab_black

In [ ]:
#client = Socrata("data.austintexas.gov", None)
#results = client.get("7d8e-dm7r", limit=83000000)

# Convert to pandas DataFrame
#df = pd.DataFrame.from_records(results)

In [3]:
df = pd.read_csv("../data/Shared_Micromobility_Vehicle_Trips.csv", low_memory=False)

In [4]:
df.head()

,ID,Device ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Year,Census Tract Start,Census Tract End
0,1d4f113d-71b2-4213-9170-1644e048d4a1,d4d91f54-5527-4bef-9ce8-24808fbfadc3,scooter,420,869.0,04/10/2018 08:00:00 PM,04/10/2018 08:00:00 PM,04/17/2019 01:58:49 AM,4,20,2,9,9,2018,48453001100,48453001100
1,05d3d08f-9960-48e3-82f6-79a375ee28b0,4f62c903-e21b-499f-9197-fd0498f25cc5,scooter,371,1037.0,04/10/2018 07:45:00 PM,04/10/2018 08:00:00 PM,04/17/2019 01:58:49 AM,4,19,2,9,9,2018,48453000603,48453000601
2,61a4992e-7dff-4142-8dd6-baa23acf561a,6a25bdcf-1f68-4778-95b4-4223c7beec15,scooter,1687,4491.0,04/23/2018 01:15:00 PM,04/23/2018 01:45:00 PM,04/17/2019 02:42:15 AM,4,13,1,9,9,2018,48453001100,48453001100
3,f1410b76-327b-479c-825e-45168034f390,b6efa85b-a836-4412-9620-77609f0e612b,scooter,524,633.0,04/27/2018 09:45:00 PM,04/27/2018 10:00:00 PM,04/17/2019 02:42:15 AM,4,21,5,9,9,2018,48453000603,48453000604
4,61250373-bece-46c2-a7c6-fb91c53eeefe,7af82887-fe34-4327-a63a-dfd11646252f,scooter,322,1114.0,04/27/2018 09:45:00 PM,04/27/2018 09:45:00 PM,04/17/2019 02:42:15 AM,4,21,5,9,9,2018,48453000500,48453000601


In [5]:
df.describe()

,Trip Duration,Trip Distance,Month,Hour,Day of Week,Year
count,9.520142e+06,9.520142e+06,9.520142e+06,9.520142e+06,9.520142e+06,9.520142e+06
mean,6.772429e+02,4.240844e+04,6.728743e+00,1.442792e+01,3.279627e+00,2.018883e+03
std,6.362652e+03,5.850044e+06,3.352879e+00,5.458836e+00,2.103152e+00,5.946058e-01
min,-8.616100e+04,-1.672411e+07,1.000000e+00,0.000000e+00,0.000000e+00,2.018000e+03
25%,2.340000e+02,4.840000e+02,4.000000e+00,1.100000e+01,1.000000e+00,2.019000e+03
50%,4.060000e+02,1.031000e+03,7.000000e+00,1.500000e+01,4.000000e+00,2.019000e+03
75%,7.530000e+02,1.931000e+03,1.000000e+01,1.800000e+01,5.000000e+00,2.019000e+03
max,1.149160e+07,2.147484e+09,1.200000e+01,2.300000e+01,6.000000e+00,2.020000e+03


In [7]:
df.shape

(9520142, 16)

In [10]:
df["Vehicle Type"].value_counts()

scooter    9070866
bicycle     449276
Name: Vehicle Type, dtype: int64

In [11]:
df["Year"].value_counts()

2019    6023897
2018    2305093
2020    1191152
Name: Year, dtype: int64

In [9]:
df.apply(lambda x: x.isnull().value_counts())

,ID,Device ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Year,Census Tract Start,Census Tract End
False,9520142.0,9520142.0,9520142.0,9520142.0,9520142.0,9520142.0,9520142.0,9520142.0,9520142.0,9520142.0,9520142.0,9520008,9520008,9520142.0,9520007,9520007
True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134,134,NaN,135,135
